In [ ]:
!huggingface-cli login
# Enter the HF access token generated from the HuggingFace account page

In [5]:
import os
import datasets
import transformers
import peft
import trl
import torch

In [4]:
EFFICIENT_TRAINING = True

In [6]:
compute_dtype = getattr(torch, "float16")
quant_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype, # Run computations in 16-bit float
    bnb_4bit_use_double_quant=False, # Use True for additional memory saving
) if EFFICIENT_TRAINING else None

In [7]:
base_model_name = "Qwen/Qwen2.5-3B-Instruct"
base_model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config if EFFICIENT_TRAINING else None,
    device_map={"": 0}
)
if EFFICIENT_TRAINING:
    base_model.config.use_cache = False
    base_model.config.pretraining_tp = 1 # A value other than 1 uses a slower, more accurate linear layer

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
model = peft.PeftModel.from_pretrained(base_model, "qwem7b_4bit_s1")

model.push_to_hub("jk23541/qwem7b_4bit_s1")

adapter_model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jk23541/qwem7b_4bit_s1/commit/895bd6db51f2133854e6117f428d2505a934ebbc', commit_message='Upload model', commit_description='', oid='895bd6db51f2133854e6117f428d2505a934ebbc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jk23541/qwem7b_4bit_s1', endpoint='https://huggingface.co', repo_type='model', repo_id='jk23541/qwem7b_4bit_s1'), pr_revision=None, pr_num=None)